# Onderzoek Openstreetmap wijken en buurten

In [15]:
# openstreetmap API package
# can be installed in conda cmd with:
# conda config --prepend channels conda-forge
# conda create -n ox --strict-channel-priority osmnx
from osmnx import geometries

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

## pak de gezondheid wijken en buurten data

In [16]:
gezondheids_df = pd.read_csv('data/alkmaar-gezondheid-wijk_en_buurt.csv')
display(gezondheids_df)

,Leeftijd,Marges,Perioden,WijkenEnBuurten,Gemeentenaam,SoortRegio,Regioaanduiding/Codering (),ErvarenGezondheidGoed,Ondergewicht,NormaalGewicht,Overgewicht,ErnstigOvergewicht,VoldoetAanAlcoholRichtlijn,Drinker,ZwareDrinker,OvermatigeDrinker
0,18 jaar of ouder,Waarde,2020,Alkmaar,Alkmaar,Gemeente,GM0361,78.5,2.1,49.5,48.6,14.7,41.1,81.2,8.9,6.8
1,18 jaar of ouder,Waarde,2020,Zuid,Alkmaar,Wijk,WK036101,77.8,2.2,51.8,46,13.7,41.2,81,8.8,7.1
2,18 jaar of ouder,Waarde,2020,Kooimeer,Alkmaar,Buurt,BU03610100,70.8,2.4,46.5,51.9,18.1,53,67,8.1,5.8
3,18 jaar of ouder,Waarde,2020,Dillenburg en Stadhouderskwartier,Alkmaar,Buurt,BU03610101,74.1,2,48.1,50.3,15.6,45.3,76.9,8.3,6.3
4,18 jaar of ouder,Waarde,2020,Staatsliedenkwartier en Landstraten,Alkmaar,Buurt,BU03610102,75,2.2,48.6,48.1,15.5,42.3,78.4,9.6,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,18 jaar of ouder,Waarde,2020,Starnmeer,Alkmaar,Buurt,BU03611005,86.1,2,50.7,48.2,10.6,35.7,87.6,8.8,7.5
74,18 jaar of ouder,Waarde,2020,Markenbinnen,Alkmaar,Buurt,BU03611006,84,1.9,50,48.8,12.6,32.2,88.7,9.4,7.9
75,18 jaar of ouder,Waarde,2020,Vroonermeer,Alkmaar,Wijk,WK036111,83.7,2,51.7,46.8,13,39.6,83.8,8,5.7
76,18 jaar of ouder,Waarde,2020,Vroonermeer-Zuid,Alkmaar,Buurt,BU03611100,83.1,2,51.4,47.1,13.1,40.6,83.3,8,6


### data opschonen

In [17]:
# Sum van alle Nan values per coloumn
print(f'Voordat we de null-waardes droppen hebben we: {len(gezondheids_df)}  in de dataset')
print(gezondheids_df.isnull().sum())
gezondheids_df.dropna(inplace=True)
# veel wijken en buurten bevatten als null waarde bij de gezondheidsdata een '.'
gezondheids_df = gezondheids_df.drop(gezondheids_df[gezondheids_df.Ondergewicht == '.'].index)
print(f'Nadat we de null-waardes droppen hebben we: {len(gezondheids_df)} in de  dataset')

Voordat we de null-waardes droppen hebben we: 78  in de dataset
Leeftijd                       0
Marges                         0
Perioden                       0
WijkenEnBuurten                0
Gemeentenaam                   0
SoortRegio                     0
Regioaanduiding/Codering ()    0
ErvarenGezondheidGoed          0
Ondergewicht                   0
NormaalGewicht                 0
Overgewicht                    0
ErnstigOvergewicht             0
VoldoetAanAlcoholRichtlijn     0
Drinker                        0
ZwareDrinker                   0
OvermatigeDrinker              0
dtype: int64
Nadat we de null-waardes droppen hebben we: 77 in de  dataset


## pak Openstreetmap data voor alcoholgebruik en obesitas.

Maak een lijst met alle query's voor het zoeken naar wijken en buurten.

In [18]:
places = []

for ind_r, row_b in gezondheids_df.iterrows():
    if str(row_b['SoortRegio']) in ['Gemeente', 'Land']:
        places.append([str(row_b['Gemeentenaam']) + ', Nederland', row_b['SoortRegio']])
    else:
        places.append([str(row_b['WijkenEnBuurten']) + ', ' + str(row_b['Gemeentenaam']) + ', Nederland', row_b['SoortRegio']])


Kies welke facalliteiten gezocht moeten worden.

In [19]:
alcoholgebruik_tags = {'amenity': ['pub', 'bar']}
obesitas_tags = {'amenity': ['fast_food']}

zoek naar alle wijken en buurten voor de gekozen tags.

In [20]:
# lijsten om de data in op te slaan.
bar_data = []
fastfood_data = []
# var om het aantal errors te tellen van niet gevonden plaatsen
errors = 0

# loop door de wijken en buurten heen en zoek naar de voorzieningen.
print('start searching')
for place in places:
    
    # zoek in gemeente en land
    if place[1] in ['Gemeente', 'Land']:
        try:
            # zoek naar bars in de grenzen
            bar = geometries.geometries_from_place(place[0], alcoholgebruik_tags)
    
            # zoek naar fastfood restaurants in de grenzen
            fastfood = geometries.geometries_from_place(place[0], obesitas_tags)
            
        except ValueError:
            bar_data.append(None)
            fastfood_data.append(None)
            errors += 1
            continue

    # zoek in wijken en buurten
    else:
        try:
            # zoek naar bars in een afstand vanaf de wijk of buurt
            bar = geometries.geometries_from_address(place[0], alcoholgebruik_tags, dist=500)
        
            # zoek naar fastfood restaurants in een afstand vanaf de wijk of buurt
            fastfood = geometries.geometries_from_address(place[0], obesitas_tags, dist=500)
            
        except ValueError:
            bar_data.append(None)
            fastfood_data.append(None)
            errors += 1
            continue
    
    bar_data.append(len(bar))
    fastfood_data.append(len(fastfood))
    
    per = places.index(place) / len(places) * 100
    print('%d procent gezocht ; ' % per + 'errors: ' + str(errors), end='\r')


start searching


D:\anaconda\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
D:\anaconda\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [21]:
print(bar_data)
print(len(gezondheids_df))

[36, 0, 0, None, None, 0, 2, None, 0, 1, None, 0, 1, None, None, 1, None, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, 0, None, 0, None, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, None, 0, 0, None, 31, 30, 25, 7, 5, 0, 0, None, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
77


In [22]:
gezondheids_df['n_bars'] = bar_data
gezondheids_df['n_fastfood'] = fastfood_data

In [23]:
display(gezondheids_df)

,Leeftijd,Marges,Perioden,WijkenEnBuurten,Gemeentenaam,SoortRegio,Regioaanduiding/Codering (),ErvarenGezondheidGoed,Ondergewicht,NormaalGewicht,Overgewicht,ErnstigOvergewicht,VoldoetAanAlcoholRichtlijn,Drinker,ZwareDrinker,OvermatigeDrinker,n_bars,n_fastfood
0,18 jaar of ouder,Waarde,2020,Alkmaar,Alkmaar,Gemeente,GM0361,78.5,2.1,49.5,48.6,14.7,41.1,81.2,8.9,6.8,36.0,55.0
1,18 jaar of ouder,Waarde,2020,Zuid,Alkmaar,Wijk,WK036101,77.8,2.2,51.8,46,13.7,41.2,81,8.8,7.1,0.0,0.0
2,18 jaar of ouder,Waarde,2020,Kooimeer,Alkmaar,Buurt,BU03610100,70.8,2.4,46.5,51.9,18.1,53,67,8.1,5.8,0.0,1.0
3,18 jaar of ouder,Waarde,2020,Dillenburg en Stadhouderskwartier,Alkmaar,Buurt,BU03610101,74.1,2,48.1,50.3,15.6,45.3,76.9,8.3,6.3,NaN,NaN
4,18 jaar of ouder,Waarde,2020,Staatsliedenkwartier en Landstraten,Alkmaar,Buurt,BU03610102,75,2.2,48.6,48.1,15.5,42.3,78.4,9.6,7.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,18 jaar of ouder,Waarde,2020,Starnmeer,Alkmaar,Buurt,BU03611005,86.1,2,50.7,48.2,10.6,35.7,87.6,8.8,7.5,0.0,0.0
74,18 jaar of ouder,Waarde,2020,Markenbinnen,Alkmaar,Buurt,BU03611006,84,1.9,50,48.8,12.6,32.2,88.7,9.4,7.9,0.0,0.0
75,18 jaar of ouder,Waarde,2020,Vroonermeer,Alkmaar,Wijk,WK036111,83.7,2,51.7,46.8,13,39.6,83.8,8,5.7,0.0,0.0
76,18 jaar of ouder,Waarde,2020,Vroonermeer-Zuid,Alkmaar,Buurt,BU03611100,83.1,2,51.4,47.1,13.1,40.6,83.3,8,6,0.0,0.0


In [24]:
# Sum van alle Nan values per coloumn
print(f'Voordat we de null-waardes droppen hebben we: {len(gezondheids_df)}  in de dataset')
print(gezondheids_df.isnull().sum())
gezondheids_df.dropna(inplace=True)
# veel wijken en buurten bevatten als null waarde bij de gezondheidsdata een '.'
gezondheids_df = gezondheids_df.drop(gezondheids_df[gezondheids_df.Ondergewicht == '.'].index)
print(f'Nadat we de null-waardes droppen hebben we: {len(gezondheids_df)} in de  dataset')

Voordat we de null-waardes droppen hebben we: 77  in de dataset
Leeftijd                        0
Marges                          0
Perioden                        0
WijkenEnBuurten                 0
Gemeentenaam                    0
SoortRegio                      0
Regioaanduiding/Codering ()     0
ErvarenGezondheidGoed           0
Ondergewicht                    0
NormaalGewicht                  0
Overgewicht                     0
ErnstigOvergewicht              0
VoldoetAanAlcoholRichtlijn      0
Drinker                         0
ZwareDrinker                    0
OvermatigeDrinker               0
n_bars                         14
n_fastfood                     14
dtype: int64
Nadat we de null-waardes droppen hebben we: 63 in de  dataset


In [25]:
populatie_df = pd.read_csv('data/alkmaar-populatie-wijk_en_buurt.csv')
gezondheids_df['populatie'] = populatie_df['Aantal']
display(gezondheids_df)

,Leeftijd,Marges,Perioden,WijkenEnBuurten,Gemeentenaam,SoortRegio,Regioaanduiding/Codering (),ErvarenGezondheidGoed,Ondergewicht,NormaalGewicht,Overgewicht,ErnstigOvergewicht,VoldoetAanAlcoholRichtlijn,Drinker,ZwareDrinker,OvermatigeDrinker,n_bars,n_fastfood,populatie
0,18 jaar of ouder,Waarde,2020,Alkmaar,Alkmaar,Gemeente,GM0361,78.5,2.1,49.5,48.6,14.7,41.1,81.2,8.9,6.8,36.0,55.0,109436
1,18 jaar of ouder,Waarde,2020,Zuid,Alkmaar,Wijk,WK036101,77.8,2.2,51.8,46,13.7,41.2,81,8.8,7.1,0.0,0.0,14510
2,18 jaar of ouder,Waarde,2020,Kooimeer,Alkmaar,Buurt,BU03610100,70.8,2.4,46.5,51.9,18.1,53,67,8.1,5.8,0.0,1.0,2135
5,18 jaar of ouder,Waarde,2020,Oud-Rochdale,Alkmaar,Buurt,BU03610103,68.2,2.1,46.1,51.7,18.8,47.3,72.6,9.9,7.7,0.0,1.0,815
6,18 jaar of ouder,Waarde,2020,Emmakwartier,Alkmaar,Buurt,BU03610104,80.4,2.4,54.1,42.3,11.2,36.4,85.8,9.1,7.8,2.0,6.0,1640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,18 jaar of ouder,Waarde,2020,Starnmeer,Alkmaar,Buurt,BU03611005,86.1,2,50.7,48.2,10.6,35.7,87.6,8.8,7.5,0.0,0.0,205
74,18 jaar of ouder,Waarde,2020,Markenbinnen,Alkmaar,Buurt,BU03611006,84,1.9,50,48.8,12.6,32.2,88.7,9.4,7.9,0.0,0.0,325
75,18 jaar of ouder,Waarde,2020,Vroonermeer,Alkmaar,Wijk,WK036111,83.7,2,51.7,46.8,13,39.6,83.8,8,5.7,0.0,0.0,4115
76,18 jaar of ouder,Waarde,2020,Vroonermeer-Zuid,Alkmaar,Buurt,BU03611100,83.1,2,51.4,47.1,13.1,40.6,83.3,8,6,0.0,0.0,3130


In [30]:
# lijst om nieuwe bar data in te bewaren
bar_pop = []
fastfood_pop = []

for ind_r, row_b in gezondheids_df.iterrows(): 
    if row_b['n_bars'] != 0:
        # deel populatie door het aantal bars
        bar_pop.append(row_b['n_bars'] / int(row_b['populatie']))
    else:
        # print de gemeenten zonder bars ter info
        print(row_b['WijkenEnBuurten'])
        bar_pop.append(row_b['populatie'])

        
for ind_r, row_b in gezondheids_df.iterrows(): 
    if row_b['n_fastfood'] != 0:
        # deel populatie door het aantal bars
        fastfood_pop.append(row_b['n_fastfood'] / int(row_b['populatie']))
    else:
        # print de gemeenten zonder bars ter info
        print(row_b['WijkenEnBuurten'])
        fastfood_pop.append(row_b['populatie'])

gezondheids_df['bar_pop'] = bar_pop
gezondheids_df['fastfood_pop'] = fastfood_pop

Zuid
Kooimeer
Oud-Rochdale
Oranjepark
Burgemeesterskwartier
Oudorperpolder-Zuid
Oudorperpolder-Midden
De Nollen
Beverkoog
Overdie
Oud-Overdie
Oosterhout
Overdie-Oost
Overdie-West
Boekelermeer-Noord
West
Bergerwegkwartier
Bergermeer
Huiswaard
Huiswaard-1-Zuid
Muiderwaard
Huiswaard-2-West
Huiswaard-2-Oost
De Mare
't Rak-Noord
De Horn-Noord
De Horn-Zuid
De Mare
Daalmeer/Koedijk
Daalmeer-Zuidoost
Daalmeer-Zuidwest
Daalmeer-Noordwest
Daalmeer-Noordoost
Schermer
Oterleek
Stompetoren
Schermerhorn
Driehuizen
Grootschermer
Noordeinde
West-Graftdijk
Graft
De Rijp
Oost-Graftdijk
Starnmeer
Markenbinnen
Vroonermeer
Vroonermeer-Zuid
Vroonermeer-Noord
Zuid
Oranjepark
Burgemeesterskwartier
De Nollen
Boekelermeer-Noord
West
Bergerwegkwartier
Bergermeer
Huiswaard-1-Zuid
't Rak-Noord
De Horn-Noord
Daalmeer-Noordwest
Daalmeer-Noordoost
Schermer
Oterleek
Stompetoren
Schermerhorn
Zuidschermer
Driehuizen
Grootschermer
Graft-De Rijp
Noordeinde
West-Graftdijk
Graft
De Rijp
Oost-Graftdijk
Starnmeer
Markenbinnen

In [31]:
gezondheids_df.Ondergewicht = gezondheids_df.Ondergewicht.astype(float)
gezondheids_df.ErvarenGezondheidGoed = gezondheids_df.ErvarenGezondheidGoed.astype(float)
gezondheids_df.NormaalGewicht = gezondheids_df.NormaalGewicht.astype(float)
gezondheids_df.Overgewicht = gezondheids_df.Overgewicht.astype(float)
gezondheids_df.ErnstigOvergewicht = gezondheids_df.ErnstigOvergewicht.astype(float)
gezondheids_df.VoldoetAanAlcoholRichtlijn = gezondheids_df.VoldoetAanAlcoholRichtlijn.astype(float)
gezondheids_df.ZwareDrinker = gezondheids_df.ZwareDrinker.astype(float)
gezondheids_df.OvermatigeDrinker = gezondheids_df.OvermatigeDrinker.astype(float)

In [32]:
gezondheids_df.corr()

,Perioden,ErvarenGezondheidGoed,Ondergewicht,NormaalGewicht,Overgewicht,ErnstigOvergewicht,VoldoetAanAlcoholRichtlijn,ZwareDrinker,OvermatigeDrinker,n_bars,n_fastfood,populatie,bar_pop,fastfood_pop
Perioden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ErvarenGezondheidGoed,NaN,1.000000,-0.144460,0.705629,-0.689851,-0.912174,-0.911729,0.331794,0.433003,0.090882,-0.008871,-0.049610,-0.181671,0.117134
Ondergewicht,NaN,-0.144460,1.000000,0.310793,-0.374757,0.013721,0.152104,0.193576,-0.203510,0.321734,0.284319,0.074919,0.070781,0.032117
NormaalGewicht,NaN,0.705629,0.310793,1.000000,-0.986873,-0.853235,-0.667522,0.313328,0.252898,0.364212,0.228105,-0.009950,-0.176481,0.122891
Overgewicht,NaN,-0.689851,-0.374757,-0.986873,1.000000,0.845405,0.671572,-0.355596,-0.276986,-0.402994,-0.262240,-0.004617,0.163903,-0.134420
ErnstigOvergewicht,NaN,-0.912174,0.013721,-0.853235,0.845405,1.000000,0.877250,-0.313000,-0.491343,-0.186614,-0.067211,0.057766,0.246875,-0.101285
VoldoetAanAlcoholRichtlijn,NaN,-0.911729,0.152104,-0.667522,0.671572,0.877250,1.000000,-0.613351,-0.676918,-0.233130,-0.097164,0.049030,0.226259,-0.093270
ZwareDrinker,NaN,0.331794,0.193576,0.313328,-0.355596,-0.313000,-0.613351,1.000000,0.657735,0.489723,0.329863,0.015898,-0.120268,0.058861
OvermatigeDrinker,NaN,0.433003,-0.203510,0.252898,-0.276986,-0.491343,-0.676918,0.657735,1.000000,0.266918,0.139619,-0.058824,-0.233376,0.006811
n_bars,NaN,0.090882,0.321734,0.364212,-0.402994,-0.186614,-0.233130,0.489723,0.266918,1.000000,0.925075,0.572363,-0.204482,-0.142078
